<a href="https://colab.research.google.com/github/ogunerkutay/AICREW/blob/main/AICrew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 0: Required Installs


In [ ]:
!pip install python-dotenv
!pip install gitpython
!pip install langchain
!pip install langchain-community
!pip install langchain_google_genai
!pip install faiss-gpu
!pip install wikipedia
!pip install crewai

# Part 1: Environment Setup and Basic Imports


In [2]:
from dotenv import load_dotenv
import os
import git

def get_api_key(key_name):
    """Get API key from Colab secrets or environment variables based on the environment."""
    try:
        # First, try to fetch from Colab secrets if available
        from google.colab import userdata
        return userdata.get(key_name)
    except ImportError:
        # If not in Colab, load from .env file
        from dotenv import load_dotenv
        load_dotenv()
        return os.environ.get(key_name)

# API Keys
gemini_api_key = get_api_key("GEMINI_API_KEY")
google_api_key = get_api_key("GOOGLE_API_KEY")


# Get and print the current working directory
current_directory = os.getcwd()
print("Current working directory:", current_directory)



Current working directory: /content


# Part 2: Configuration and Initializations

In [9]:
# Part 2: Configuration and Initializations
from langchain.globals import set_debug
from langchain.embeddings import CacheBackedEmbeddings
from langchain.globals import set_debug
from langchain.storage import LocalFileStore
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from tempfile import TemporaryDirectory

# Set debug mode
set_debug(False)

# To load gemini (this api is for free: https://makersuite.google.com/app/apikey)
llm = ChatGoogleGenerativeAI(model="gemini-pro", verbose=True, temperature=0.1, google_api_key=gemini_api_key, convert_system_message_to_human=True)

# Create a local file store for caching embeddings
store = LocalFileStore("./cache/")

core_embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key)

# Setup embedder with cache
embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model,
    store,
    namespace = core_embeddings_model.model
)


# Part 3: Document Processing Functions

In [4]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

def load_and_process_document(file_path, chunk_size=1000, chunk_overlap=0):
    """Load a document, split it into chunks, and process it for vector store."""
    # Load a document
    loader = TextLoader(file_path)
    documents = loader.load()

    # Split the document into chunks
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return text_splitter.split_documents(documents)

# Check if running in Google Colab
running_in_colab = 'google.colab' in str(get_ipython())

if running_in_colab:
    # If running in Colab, mount Google Drive and set the directory there
    #from google.colab import drive
    #drive.mount('/content/drive')

    # Define the Google Drive directory for documents
    document_directory = '/content/drive/My Drive/rag/game.txt'
else:
    # If running locally, use the local file system
    document_directory = "/mnt/c/Users/ogune/OneDrive/Desktop/MYAI/merai/rag/game.txt"

# Load documents from the directory
document_texts = load_and_process_document(document_directory)

# Part 4: RetrievalQA Chain

In [5]:
from langchain_community.vectorstores import FAISS
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import RetrievalQA

def create_vector_store(documents_texts, embedder):
    """Create and populate a FAISS vector store."""
    if documents_texts:
        from langchain_community.vectorstores import FAISS
        return FAISS.from_documents(documents_texts, embedder)
    else:
        print("No documents provided for vector store creation.")
        return None

# Create and populate the vector store
vectorstore = create_vector_store(document_texts, embedder)

# Instantiate a retriever
retriever = vectorstore.as_retriever()

handler = StdOutCallbackHandler()
qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)

# Part 5: Tools Initialization

In [6]:
from langchain.tools import tool
from langchain.agents.load_tools import load_tools
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, SearxSearchWrapper

# Define a custom tool for the agent using the RAG mechanism
@tool
def rag_tool(query: str) -> str:
    """
    A tool to perform Retrieval Augmented Generation.

    Args:
        query (str): The query string for which to generate a response.

    Returns:
        str: The generated response.
    """
    response = qa_with_sources_chain({"query": query})
    return response['result']

# Initialize File Management Toolkit
working_directory = TemporaryDirectory()
print("Temporary directory path:", working_directory.name)
file_tool = FileManagementToolkit(
    root_dir=str(working_directory.name),
    selected_tools=["read_file", "write_file", "list_directory"]
    )

file_tools = file_tool.get_tools()

# Define the SearxNG search tool
@tool
def searxng_search_tool(query: str) -> str:
    """
    A tool to perform a search using SearxNG.

    Args:
        query (str): The search query string.

    Returns:
        str: The search results.
    """
    # Replace 'your_searxng_host' with the host of your SearxNG instance
    searx_host = 'http://127.0.0.1:8888'
    search = SearxSearchWrapper(searx_host=searx_host)

    results = search.run(query)
    return results

# Initialize Wikipedia Tool
wikipedia_api_wrapper = WikipediaAPIWrapper()
wikipedia_tool = WikipediaQueryRun(api_wrapper=wikipedia_api_wrapper)

# Initialize Requests Tools
request_tools = load_tools(["requests_all"])

# Loading Human Tools
human_tool = load_tools(["human"])

# Define the Project Supervisor tool
@tool
def project_supervisor_tool(commit_message: str, content: str) -> str:
    """
    A tool to simulate the review and approval of code changes by a project supervisor.
    - commit_message: The message associated with the commit.
    - content: The content of the code changes.
    Returns approval status as a string.
    """
    # Simulate review process
    print(f"Reviewing changes: {commit_message}\nCode Changes:\n{content}")
    approval = input("Do you approve these changes? (yes/no): ").strip().lower()
    if approval == 'yes':
        return "Changes approved"
    else:
        return "Changes not approved"


# Define Code Execution Tool
@tool
def execute_code(code: str, file_path: str) -> str:
    """
    Executes the given Python code and returns its output.

    Args:
        code (str): Python code to execute.
        file_path (str): Path to the file where the code will be written before execution.

    Returns:
        str: The standard output or error from executing the code.
    """
    with open(file_path, 'w') as file:
        file.write(code)
    try:
        output = subprocess.run(['python', file_path], capture_output=True, text=True, check=True)
        return output.stdout
    except subprocess.CalledProcessError as e:
        return e.stderr

# Define Static Analysis Tool
@tool
def static_analysis(code: str) -> str:
    """
    Performs static analysis on the provided code to check for syntax errors.

    Args:
        code (str): Python code to analyze.

    Returns:
        str: Results of the static analysis.
    """
    # Use a static analysis library or implement your own logic
    # Placeholder for static analysis result
    return "No syntax errors found"

# Define Security Vulnerability Scanner Tool
@tool
def security_scan(code: str) -> str:
    """
    Scans the provided code for known security vulnerabilities.

    Args:
        code (str): Python code to scan.

    Returns:
        str: Results of the security scan.
    """
    # Use a security scanning library or API
    # Placeholder for security scan result
    return "No security vulnerabilities found"

# Define Output Assessment Tool
@tool
def assess_output(code: str, expected_output: str) -> str:
    """
    Compares the actual output of the code with the expected output.

    Args:
        code (str): Python code to execute.
        expected_output (str): Expected output of the code.

    Returns:
        str: Assessment result indicating if the output matches the expected output.
    """
    # Logic to compare actual and expected output
    # Placeholder for output assessment result
    return "Output matches expected result"

# Define Bandit Security Analysis Tool
@tool
def bandit_analysis(file_path: str) -> str:
    """
    Runs Bandit security analysis on the Python file at the given path.

    Args:
        file_path (str): Path to the Python file to analyze.

    Returns:
        str: Results of the Bandit analysis.
    """
    result = subprocess.run(['bandit', '-r', file_path, '-f', 'text'], capture_output=True, text=True)
    return result.stdout

# Define Safety Vulnerability Scanner Tool
@tool
def safety_scan(file_path: str) -> str:
    """
    Runs a Safety vulnerability scan on the Python file at the given path.

    Args:
        file_path (str): Path to the Python file to scan.

    Returns:
        str: Results of the Safety scan.
    """
    result = subprocess.run(['safety', 'check', '--file', file_path, '--full-report'], capture_output=True, text=True)
    return result.stdout

combined_tools =  [wikipedia_tool, rag_tool, project_supervisor_tool, execute_code, static_analysis, security_scan, assess_output, bandit_analysis, safety_scan] + file_tools + request_tools + human_tool


Temporary directory path: /tmp/tmpnf9052tk


# Part 6: Agents and Crew Initialization

In [7]:
from crewai import Agent, Task, Crew, Process

# Define the different roles for the software development team
# Create a software development team
product_owner = Agent(
    role="Product Owner",
    goal="Define the product vision, prioritize features, and guide the development process.",
    backstory="You are responsible for defining the direction of the mobile game and ensuring it meets user expectations.",
    llm=llm,
    tools=[*combined_tools]
)

devops_engineer = Agent(
    role="DevOps Engineer",
    goal="Set up deployment pipelines and manage the infrastructure for the application.",
    backstory="As a DevOps expert, you will ensure smooth deployment and operation of the applications",
    llm=llm,
    tools=[*combined_tools]
)

database_expert = Agent(
    role="Database Expert",
    goal="Design and optimize the database structure for the application.",
    backstory="As a database expert, you will create a robust database schema and optimize queries for efficient data retrieval.",
    llm=llm,
    tools=[*combined_tools]
)

backend_developer = Agent(
    role="Backend Developer",
    goal="Build the backend logic and APIs of the application using C# and .NET Core.",
    backstory="Leverage your expertise in C# and .NET Core to develop a scalable and efficient backend system.",
    llm=llm,
    tools=[*combined_tools]
)

middleware_expert = Agent(
    role="Middleware Developer",
    goal="Develop middleware components for communication between the frontend and backend systems.",
    backstory="Your role is crucial in ensuring smooth data flow between the frontend and backend components.",
    llm=llm,
    tools=[*combined_tools]
)

frontend_developer = Agent(
    role="Frontend Developer",
    goal="Create the user interface and user experience of the mobile game using Unity.",
    backstory="Utilize Unity to design and implement an engaging and user-friendly frontend for the application.",
    llm=llm,
    tools=[*combined_tools]
)

tester = Agent(
    role="QA Tester",
    goal="Conduct thorough testing to identify and report bugs in the application.",
    backstory="Your testing skills are crucial in ensuring the quality and reliability of the application.",
    llm=llm,
    tools=[*combined_tools]
)



# Define tasks for the team
tasks = [
    Task(description="Design the database schema for the mobile game.", agent=database_expert),
    Task(description="Develop middleware components for data communication.", agent=middleware_expert),
    Task(description="Implement backend logic and APIs using C# and .NET Core.", agent=backend_developer),
    Task(description="Create the user interface and user experience using Unity.", agent=frontend_developer),
    Task(description="Perform thorough testing and report any bugs found.", agent=tester),
    Task(description="Set up deployment pipelines and manage infrastructure.", agent=devops_engineer),
    Task(description="Define the product vision and prioritize features.", agent=product_owner)
]

crew = Crew(
    agents=[product_owner, devops_engineer, database_expert, backend_developer, middleware_expert, frontend_developer, tester],
    tasks=tasks,
    verbose=2,
    process=Process.sequential
)


# Part 7: Execution and Results Handling

In [11]:
import os
import shutil
from pathlib import Path
from tempfile import TemporaryDirectory
from langchain.storage import LocalFileStore

# Kick off the crew to start the development process
result = crew.kickoff()
print("######################")
print(result)

# Temporary directory setup for working
# Temporary directory setup
working_directory = TemporaryDirectory()
print("Temporary directory path:", working_directory.name)

# Check if running in Google Colab
running_in_colab = 'google.colab' in str(get_ipython())

if running_in_colab:
    # If running in Colab, mount Google Drive and set the directory there
    from google.colab import drive
    drive.mount('/content/drive')

    # Define the Google Drive directory
    google_drive_dir = '/content/drive/My Drive/results'
    Path(google_drive_dir).mkdir(parents=True, exist_ok=True)

    # Copy files to the Google Drive directory
    shutil.copytree(working_directory.name, google_drive_dir)
    print(f"Copied files to {google_drive_dir}")
else:
    # If running locally, use the local file system
    local_dir = Path("/mnt/C:/Users/ogune/OneDrive/Desktop/results")

    # Copy files to the local directory
    shutil.copytree(working_directory.name, local_dir)
    print(f"Copied files to {local_dir}")



[DEBUG]: Working Agent: Database Expert
[INFO]: Starting Task: Design the database schema for the mobile game.


KeyboardInterrupt: 